# AutoEncoder

## 例題
MNISTデータセット（手書き数字文字画像）をAEで次元削減するプログラムを作成せよ．

### STEP1：インポート
下記ライブラリをインポートする．

- torch
- torch.nn
- torch.nn.functional
- torch.optim
- torchvision
- torchvision.transforms
- numpy
- matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

### STEP2：ネットワーク定義
ネットワークを定義するクラスを記述する． 

- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
- 今回は2つの全結合層で構成する
- 全結合層は```nn.Linear```クラスを用いる


In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        # エンコーダとして用いる全結合層の定義
        # 引数1：入力サイズ = 784
        # 引数2：出力サイズ = 100
        self.fc_enc = nn.Linear(784, 100)
    
        # デコーダとして用いる全結合層の定義
        # 引数1：入力サイズ = 100
        # 引数2：出力サイズ = 784
        self.fc_dec = nn.Linear(100, 784)

    def forward(self, x):

        # 今回用いるMNISTデータセットは，
        # 次元数が(ch, h, w) = (1, 28, 28) = 784である
        # エンコーダでは784次元のデータを100次元まで圧縮している
        x = self.fc_enc(x)
        x = F.relu(x)

        # デコーダ部では100次元まで圧縮したデータを
        # 784次元まで伸長している
        x = self.fc_dec(x)

        return x

In [ ]:
# 上記MLPクラスのインスタンスを生成
autoencoder = AutoEncoder()

# 実行するデバイスにインスタンスを移動
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
autoencoder.to(device)

### STEP3：誤差関数・最適化器
誤差関数と最適化器を定義する．
- 誤差関数：```nn.MSELoss```と```nn.CrossEntropyLoss```がよく使われる
    - ```nn.MSELoss```：回帰問題でよく使う（今回はこちらを使う）
    - ```nn.CrossEntropyLoss```：分類問題でよく使う
- 最適化器：```optim.SGD```と```optim.Adam```がよく使われる
    - ```optim.SGD```：確率的勾配降下法
    - ```optim.Adam```：Adam（今回はこちらを使う）

In [ ]:
# 誤差関数（平均二乗誤差）の定義
criterion = nn.MSELoss()

# 最適化器（Adam）の定義
# 引数1：最適化するネットワークのパラメータ parameters()で取得可能
optimizer = optim.Adam(autoencoder.parameters())

### STEP4：データセット
データセットを用意する．

- MNISTデータセット（28x28ピクセルの画像）は```torchvision```から読み出す
    - Training用データ（60000枚）
    - Test用データ（10000枚）
- バッチサイズ（一度にネットワークに入力するデータ数）を指定する
  - 大→データセット全てを入力するまでにかかる繰り返し回数が減るため学習時間が短くなるが メモリの消費が大きくなる
  - 小→データセット全てを入力するまでにかかる繰り返し回数が増えるため学習時間が長くなるが メモリの消費が小さくなる


In [ ]:
# ローカル環境で動作させる場合は
#   torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# でデータセットを./dataフォルダにダウンロードできるが，Colabだとダウンロードエラーが生じることがある． 
# 代わりにデータセットを直接ダウンロードし，ダウンロードした場所を指定する対策をとっている．

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [ ]:
# 画像整形の設定
transform_mnist = transforms.Compose([
    # PyTorchではTensor型と呼ばれる配列でデータを取り扱う
    # ここではダウンロードしてきた画像をTensor型に変換する
    transforms.ToTensor(),

    # ダウンロードした画像の値域は[0, 1]だが これを[-1, 1]に整形する
    # 入力画像inputsは下記の式に基づいて変換される
    # inputs[channel] = (inputs[channel] - mean[channel]) / std[channel]
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ])

# データセットの読み込み
# 引数1：データセットのパス
# 引数2：Training用かTest用か
# 引数3：ダウンロードしてくるか否か
# 引数4：画像整形の設定
trainset_mnist = torchvision.datasets.MNIST(root='./', train=True, download=False, transform=transform_mnist)
testset_mnist = torchvision.datasets.MNIST(root='./', train=False, download=False, transform=transform_mnist)

# ミニバッチの設定
# 引数1：データセット
# 引数2：バッチサイズ
# 引数3：データをシャッフルするか否か
batchsize = 100
trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, batch_size=batchsize, shuffle=True)
testloader_mnist = torch.utils.data.DataLoader(testset_mnist, batch_size=batchsize, shuffle=False)

In [ ]:
# 取り出したデータの確認
def sample_viewer(sample_data):
    img = torchvision.utils.make_grid(sample_data, nrow=10)
    img = img / 2 + 0.5
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg)
    plt.show()

dataiter = iter(trainloader_mnist)
images, labels = dataiter.next()
sample_viewer(images)

### STEP5：ネットワーク学習
学習を下記のステップで行う．
- 入力データ・教師データをネットワークに入力
- ネットワークパラメータの微分値をリセット：```optimizer.zero_grad()```
- ネットワークの出力と教師データの誤差を計算
  - 今回は教師データとして入力データを用いる
- 誤差逆伝播：```loss.backward()```
- ネットワークパラメータの更新：```optimizer.step()```
- 以上を繰り返す（適宜ネットワークを評価）

In [ ]:
# エポック数：データセットを何周するか
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0

    # for文でTraining用データセットをループする
    for i, data in enumerate(trainloader_mnist, 0):
        # inputs：入力データ
        # labels：教師データ
        inputs, labels = data

        # 入力データが画像なので変形してAEの入力に合わせる
        inputs = torch.flatten(inputs, 1)

        # 実行するデバイスにデータを移動
        inputs = inputs.to(device)

        # 入力データを順伝播し出力を確認
        outputs = autoencoder(inputs)

        # ネットワークパラメータの微分値をリセット
        optimizer.zero_grad()

        # ネットワークの出力と教師データ（入力データ）の誤差を計算
        loss = criterion(outputs, inputs)

        # 誤差逆伝播
        loss.backward()

        # ネットワークパラメータの更新
        optimizer.step()

        # 誤差のモニタリング
        # 100回に1回
        running_loss += loss.item()
        if i % 100 == 99:
            print("[{}, {}] train loss: {}".format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


    # ネットワークのTest
    test_loss = 0.0

    # for文でTest用データセットをループする
    for i, data in enumerate(testloader_mnist, 0):
        # 入力データを順伝播しの出力を確認
        inputs, labels = data
        inputs = torch.flatten(inputs, 1)
        inputs = inputs.to(device)
        outputs = autoencoder(inputs)

        # ネットワークの出力と教師信号の誤差を計算
        test_loss += criterion(outputs, inputs).item()

    print("test loss: {}".format(test_loss / len(testloader_mnist)))

### STEP6：データの入力テスト


In [ ]:
dataiter = iter(testloader_mnist)
inputs, labels = dataiter.next()

# 入力画像
print("Input")
sample_viewer(inputs)

# 出力画像
print("Output")
outputs = autoencoder(torch.flatten(inputs, 1).to(device))
outputs = outputs.to("cpu").detach().view(100, 1, 28, 28)
sample_viewer(outputs)

## 演習
EMNISTデータセット（手書き英数字文字画像）をAEで次元削減するプログラムを作成せよ．

### STEP1：インポート

### STEP2：ネットワーク定義

### STEP3：誤差関数・最適化器

### STEP4：データセット
データセットを用意する．

- EMNISTデータセット（28x28ピクセルの画像）は```torchvision```から読み出す


In [ ]:
# 画像整形の設定
transform_emnist = transforms.Compose([
    # PyTorchではTensor型と呼ばれる配列でデータを取り扱う
    # ここではダウンロードしてきた画像をTensor型に変換する
    transforms.ToTensor(),

    # ダウンロードした画像の値域は[0, 1]だが これを[-1, 1]に整形する
    # 入力画像inputsは下記の式に基づいて変換される
    # inputs[channel] = (inputs[channel] - mean[channel]) / std[channel]
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ])

# データセットの読み込み
# 引数1：データセットのパス
# 引数2：Training用かTest用か
# 引数3：ダウンロードしてくるか否か
# 引数4：データセットの分割の設定
# 引数5：画像整形の設定
trainset_emnist = torchvision.datasets.EMNIST(root='./', train=True, download=True, split='balanced', transform=transform_emnist)
testset_emnist = torchvision.datasets.EMNIST(root='./', train=False, download=True, split='balanced', transform=transform_emnist)

# データの取り出しとミニバッチの設定
# 引数1：データセット
# 引数2：バッチサイズ
batchsize = 100
trainloader_emnist = torch.utils.data.DataLoader(trainset_emnist, batch_size=batchsize)
testloader_emnist = torch.utils.data.DataLoader(testset_emnist, batch_size=batchsize)

In [ ]:
# 取り出したデータの確認
def sample_viewer(sample_data):
    img = torchvision.utils.make_grid(sample_data, nrow=10)
    img = img / 2 + 0.5
    npimg = img.numpy()
    npimg = np.transpose(npimg, (2, 1, 0))
    plt.imshow(npimg)
    plt.show()

dataiter = iter(trainloader_emnist)
images, labels = dataiter.next()
sample_viewer(images)

### STEP5：ネットワーク学習

### STEP6：データの入力テスト